In [1]:
# set working directory so we can load data correctly (all functions at this point are not put into modules yet)
import os
os.chdir('..')
os.getcwd()

'/Users/964505/CT/ct_research'

Test History Encoder

In [ ]:
from ct.data.consolidating import HistoryEncoder

In [ ]:
import pandas as pd

raw_data = pd.DataFrame({
    "patient_id": [1, 1, 1, 2],
    "domain_ids": [
        [1, 2],     # session 1 (patient 1)
        [1],                # session 2 (patient 1)
        [2],             # session 3 (patient 1, next week)
        [1]                 # session 1 (patient 2)
    ],
    "domain_scores": [
        [0.8, 0.6],
        [0.9],
        [0.7],
        [0.4]
    ],
    # minutes since epoch (roughly one week gap between 2nd and 3rd)
    "start_time_min": [
        "2018-04-13 17:37:55",
        "2018-04-13 18:37:55",     # same week as first
        "2018-04-21 17:37:55",     # next week
        "2018-04-13 17:37:55"
    ]
})

In [4]:
encoder = HistoryEncoder(raw_data)
weekly_df = encoder.transform()

In [5]:
print("Shape:", weekly_df.shape)
print("Index:", weekly_df.index.names)
print("Columns:", list(weekly_df.columns))

Shape: (3, 7)
Index: ['patient_id', 'week_number']
Columns: ['domain_1_freq', 'domain_2_freq', 'week_start_ts', 'domain_1_avg', 'domain_2_avg', 'domain_1_inv', 'domain_2_inv']


In [6]:
print(weekly_df.reset_index())

   patient_id  week_number  domain_1_freq  domain_2_freq  \
0           1            0              2              1   
1           1            1              0              1   
2           2            0              1              0   

              week_start_ts  domain_1_avg  domain_2_avg  domain_1_inv  \
0 2018-04-13 17:37:55+00:00          0.85           0.6          0.15   
1 2018-04-20 17:37:55+00:00          0.00           0.7          0.00   
2 2018-04-13 17:37:55+00:00          0.40           0.0          0.60   

   domain_2_inv  
0           0.4  
1           0.3  
2           0.0  


In [7]:
weekly_df.loc[(1, 0)]

domain_1_freq                            2
domain_2_freq                            1
week_start_ts    2018-04-13 17:37:55+00:00
domain_1_avg                          0.85
domain_2_avg                           0.6
domain_1_inv                          0.15
domain_2_inv                           0.4
Name: (1, 0), dtype: object